In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from pickle import load

In [2]:
with open('X_test.pkl', 'rb') as X_test_file:
  X_test = load(X_test_file)

with open('y_test.pkl', 'rb') as y_test_file:
  y_test = load(y_test_file)

with open('X_train.pkl', 'rb') as X_train_file:
  X_train = load(X_train_file)

with open('y_train.pkl', 'rb') as y_train_file:
  y_train = load(y_train_file)

In [3]:
model = Sequential()

model.add(Conv2D(32,
                 (5, 5),
                 activation='relu',
                 input_shape=(96, 96, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [4]:
# Configures the model for training
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 92, 92, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 46, 46, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 46, 46, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 22, 22, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 22, 22, 64)        0

In [6]:
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
history=model.fit(X_train, y_train, epochs=1, batch_size=64, verbose="1", validation_data=(X_test, y_test))

In [7]:
model.save('handrecognition_model.keras')

In [8]:
from tensorflow import saved_model
from keras.models import load_model

saved_model.save(load_model("handrecognition_model.keras"), "../model_quantize/tmp_model")
#"/content/handrecognition_model.h5"), "tmp_model")

INFO:tensorflow:Assets written to: ../model_quantize/tmp_model/assets


INFO:tensorflow:Assets written to: ../model_quantize/tmp_model/assets


In [9]:
import tf2onnx

!python -m tf2onnx.convert --saved-model ../model_quantize/tmp_model --output "handrecognition_model.onnx"

<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2024-01-23 17:52:48,806 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-01-23 17:52:48,893 - INFO - Signatures found in model: [serving_default].
2024-01-23 17:52:48,893 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-01-23 17:52:48,893 - INFO - Output names: ['dense_1']
2024-01-23 17:52:48,960 - INFO - Using tensorflow=2.15.0, onnx=1.15.0, tf2onnx=1.16.1/d0e432
2024-01-23 17:52:48,960 - INFO - Using opset <onnx, 15>
2024-01-23 17:52:48,968 - INFO - Computed 0 values for constant folding
2024-01-23 17:52:48,979 - INFO - Optimizing ONNX model
2024-01-23 17:52:49,026 - INFO - After optimization: Cast -1 (1->0), Const +1 (11->12), Identity -2 (2->0), Reshape +1 (1->2), Transpose -11 (12->1)
2024-01-23 17:52:49,030 - INFO - 
2024

In [10]:
!zip -r tmp_model.zip ../model_quantize/tmp_model

updating: ../model_quantize/tmp_model/ (stored 0%)
updating: ../model_quantize/tmp_model/fingerprint.pb (stored 0%)
updating: ../model_quantize/tmp_model/variables/ (stored 0%)
updating: ../model_quantize/tmp_model/variables/variables.data-00000-of-00001 (deflated 8%)
updating: ../model_quantize/tmp_model/variables/variables.index (deflated 54%)
updating: ../model_quantize/tmp_model/saved_model.pb (deflated 88%)
  adding: ../model_quantize/tmp_model/assets/ (stored 0%)
